# Snek Demo

Snek is developed to improve upon the original cobra package by making it easier to access and less prone for unintended errors.

In [15]:
import cobra 
import snek
model = cobra.io.load_model('textbook')

## 1 Ensure Correct Spelling

When setting bounds of a reaction or the objective function with the main CobraPy package, no error is raised when a spelling error occurs.

In [25]:
with model:
    model.reactions.EX_glc__D_e.lower_bound = -5
    print(model.slim_optimize())
    model.reactions.EX_glc__D_e.lower_buond = -10
    # no error is raised but bound is not updated.
    print(model.slim_optimize())

0.4155977750929059
0.4155977750929059


This can happen frequently and is hard to debug. Also, the officially recommended way of setting exchange bounds needs 3 lines instead of 1.

In [23]:
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 5
    model.medium = medium
    print(model.slim_optimize())
    medium["EX_glc__D_e"] = 10
    model.medium = medium
    print(model.slim_optimize())

0.4155977750929059
0.8739215069684301


The following snek.core functions ensure correct spelling and are still one-liners for efficient coding.

In [24]:
with model:
    model = snek.set_bounds(model,'EX_glc__D_e',-5)
    print(model.slim_optimize())
    model = snek.set_bounds(model,'EX_glc__D_e',-10)
    print(model.slim_optimize())

0.4155977750929059
0.8739215069684301


The same issue can arise when trying to change the objective function of a model.

In [35]:
with model:
    print(model.slim_optimize())
    model.objctive = 'EX_etoh_e'
    # no error is raised but objective function is not updated.
    print(model.slim_optimize())
    model = snek.set_objective(model,'EX_etoh_e','max')
    print(model.slim_optimize())

0.8739215069684301
0.8739215069684301
20.0


## 2 Checking for Logical Errors